# 2020 NFL Ratings

In [1]:
import pandas as pd
import numpy as np
import random
from functools import reduce

In [2]:
# For reproducibility
np.random.seed(9)

# All 32 teams in alphabetical order
teams = ["Arizona Cardinals", "Atlanta Falcons", "Baltimore Ravens", "Buffalo Bills", "Carolina Panthers", "Chicago Bears", "Cincinnati Bengals",
        "Cleveland Browns", "Dallas Cowboys", "Denver Broncos", "Detroit Lions", "Green Bay Packers", "Houston Texans", "Indianapolis Colts",
        "Jacksonville Jaguars", "Kansas City Chiefs", "Miami Dolphins", "Minnesota Vikings", "New England Patriots", "New Orleans Saints",
        "New York Giants", "New York Jets", "Las Vegas Raiders", "Philadelphia Eagles", "Pittsburgh Steelers", "Los Angeles Rams", "Los Angeles Chargers",
        "San Francisco 49ers", "Seattle Seahawks", "Tampa Bay Buccaneers", "Tennessee Titans", "Washington Football Team"]

# Generates list of 32 numbers in range [-10, 10] with average 0. To be used for initial trial ratings
# https://stackoverflow.com/questions/39435481/how-to-generate-numbers-in-range-with-specific-average-with-python
def gen_avg(expected_avg = 0, n = 32, a = -10, b = 10):
    while True:
        l = [random.randint(a, b) for i in range(n)]
        avg = reduce(lambda x, y: x + y, l) / len(l)

        if avg == expected_avg:
            return l
        
ratings = gen_avg()
off_ratings = gen_avg()
def_ratings = gen_avg()

# Put teams and ratings into one data frame
teams = pd.DataFrame(data = {"Team": teams, "ID": np.arange(32), "Rating": ratings,
                             "Off._Rating": off_ratings, "Def._Rating": def_ratings})
display(teams.head())
teams.to_csv("./data/team_ids_and_ratings.csv")  # Export for Excel use

,Team,ID,Rating,Off._Rating,Def._Rating
0,Arizona Cardinals,0,8,-5,-8
1,Atlanta Falcons,1,-3,-4,-2
2,Baltimore Ravens,2,5,-9,-4
3,Buffalo Bills,3,-7,-7,9
4,Carolina Panthers,4,-7,4,-4


In [3]:
# Scraping regular season results
url = 'https://www.pro-football-reference.com/years/2020/week_1.htm'
df = pd.read_html(url)

# Cleaning
game_results = pd.concat(df[0:32:2], ignore_index = False)  # Gets list of game results (a list of data frames) and concats them into one
game_results = game_results.drop(2, axis = 1).drop(labels = 0, axis = 0)  # Removes useless 3rd col, drops date rows
game_results.head(6)

,0,1
1,Houston Texans,20
2,Kansas City Chiefs,34
1,New York Jets,17
2,Buffalo Bills,27
1,Seattle Seahawks,38
2,Atlanta Falcons,25


In [4]:
# Nice solution to get the teams to go side-by-side: 
# https://stackoverflow.com/questions/57763470/pandas-get-second-row-and-put-it-at-the-end-of-first-row-and-automatically-cre

game_results = pd.DataFrame([y.values.ravel() for x, y in game_results.groupby(np.arange(len(game_results)) // 2)])
game_results = (game_results.loc[:, [2, 0, 3, 1]]
                .rename({2: "Home Team", 0: "Away Team", 3: "Home Score", 1: "Away Score"}, axis = 1)
                .astype({"Home Score": "int64", "Away Score": "int64"}))  # Re-order and rename columns, change score cols to ints


In [5]:
# Defines Margin as the home score - away score. A negative means the away team won; 0 a tie
game_results["Margin"] = game_results["Home Score"] - game_results["Away Score"]
game_results.head()

,Home Team,Away Team,Home Score,Away Score,Margin
0,Kansas City Chiefs,Houston Texans,34,20,14
1,Buffalo Bills,New York Jets,27,17,10
2,Atlanta Falcons,Seattle Seahawks,25,38,-13
3,Washington Football Team,Philadelphia Eagles,27,17,10
4,Baltimore Ravens,Cleveland Browns,38,6,32


In [6]:
# Function that performs cleaning; returns data frame of all requested 2020 games
# Takes url_string for a given week and an appropriate_idx, which accounts for how many teams played that given week, making scraping work properly
def clean_scores(url_string, appropriate_idx):
    df = pd.read_html(url_string)
    
    # Cleaning
    game_results = pd.concat(df[0:appropriate_idx:2], ignore_index = False)  # Gets list of game results (a list of data frames) and concats them into one
    game_results = game_results.drop(2, axis = 1).drop(labels = 0, axis = 0)  # Removes useless 3rd col, drops date rows
    
    game_results = pd.DataFrame([y.values.ravel() for x, y in game_results.groupby(np.arange(len(game_results)) // 2)])
    game_results = (game_results.loc[:, [2, 0, 3, 1]]
                    .rename({2: "Home Team", 0: "Away Team", 3: "Home Score", 1: "Away Score"}, axis = 1)
                    .astype({"Home Score": "int64", "Away Score": "int64"}))  # Re-order and rename columns, change score cols to ints
    
    # Defines Margin as the home score - away score. A negative means the away team won; 0 a tie
    game_results["Margin"] = game_results["Home Score"] - game_results["Away Score"]
    
    # Return game_results data frame
    return game_results

In [7]:
# Call clean_scores() on all 18 weeks of the season (Round 1 of playoffs has been completed as of 01/12/20)

# CHANGE THIS NUMBER TO THE NUMBER OF COMPLETED WEEKS
num_weeks = 18  # As of 01/12/20, 18 weeks (17 regular season weeks + 1 playoff week) have been completed

# CHANGE if necessary: this is a list of how many teams participated each week. 2020 was abnormal due to COVID
appropriate_idx = [32, 32, 32, 30, 28, 28, 28, 28, 28, 28, 28, 32, 30, 32, 32, 32, 32, 12]  # The 12 at the end is start of playoffs

# Container for data frames
scores = []

for i in range(1, num_weeks + 1):
    url = 'https://www.pro-football-reference.com/years/2020/week_' + str(i) + '.htm'
    week_i_scores = clean_scores(url, appropriate_idx[i - 1])
    scores.append(week_i_scores)

In [8]:
# Concat scores into one data frame
scores = pd.concat(scores, axis = 0)
scores["Game #"] = list(np.arange(1, len(scores) + 1))
scores.head()

,Home Team,Away Team,Home Score,Away Score,Margin,Game #
0,Kansas City Chiefs,Houston Texans,34,20,14,1
1,Buffalo Bills,New York Jets,27,17,10,2
2,Atlanta Falcons,Seattle Seahawks,25,38,-13,3
3,Washington Football Team,Philadelphia Eagles,27,17,10,4
4,Baltimore Ravens,Cleveland Browns,38,6,32,5


In [9]:
# Assures we collected the exactly correct number of games
scores.shape[0] == (np.array(appropriate_idx) / 2).sum()

True

In [10]:
# Merge with teams df to get team ID numbers. Requires 2 merges. Then some straightforward cleaning
scores_merged = (scores.merge(teams, how = "inner", left_on = "Home Team", right_on = "Team", suffixes = ("_Home", "_Away"))
                .merge(teams, how = "inner", left_on = "Away Team", right_on = "Team", suffixes = ("_Home", "_Away"))
                .drop(["Team_Home", "Team_Away", "Rating_Home", "Off._Rating_Home", "Def._Rating_Home",
                       "Rating_Away", "Off._Rating_Away", "Def._Rating_Away"], axis = 1)
                .iloc[:, [5, 0, 1, 6, 7, 2, 3, 4]]
                .sort_values(by = "Game #", axis = 0)
                .reset_index(drop = True))
scores_merged.head()

,Game #,Home Team,Away Team,ID_Home,ID_Away,Home Score,Away Score,Margin
0,1,Kansas City Chiefs,Houston Texans,15,12,34,20,14
1,2,Buffalo Bills,New York Jets,3,21,27,17,10
2,3,Atlanta Falcons,Seattle Seahawks,1,28,25,38,-13
3,4,Washington Football Team,Philadelphia Eagles,31,23,27,17,10
4,5,Baltimore Ravens,Cleveland Browns,2,7,38,6,32


In [11]:
scores_merged.tail()  # Check .tail() to see if we got playoff games

,Game #,Home Team,Away Team,ID_Home,ID_Away,Home Score,Away Score,Margin
257,258,Seattle Seahawks,Los Angeles Rams,28,25,20,30,-10
258,259,Washington Football Team,Tampa Bay Buccaneers,31,29,23,31,-8
259,260,Tennessee Titans,Baltimore Ravens,30,2,13,20,-7
260,261,New Orleans Saints,Chicago Bears,19,5,21,9,12
261,262,Pittsburgh Steelers,Cleveland Browns,24,7,37,48,-11


Looks good!

In [12]:
# Export to csv so we can use Excel solver...
scores_merged.to_csv("./data/nfl_game_data.csv", index = False)  